In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom')

In [ ]:
!pip install torchmetrics
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam2.git' # colab
!mkdir -p ../checkpoints/
!wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import torch
import numpy as np
import random
import os
import shutil
import copy

from parameters import Parameters
from deepfake_datasets import LOCALIZATION_DATASET_PATHS, DETECTION_DATASET_PATHS
from deepfake_datasets.datasets import get_dataloader
from models.declip import get_model
from models.declip_detection import get_detection_model
from models.sam_localization import get_sam_model
from models.two_step_model import TwoStepModel
from train.validate import validate_detection, validate_fully_supervised_localization, validate_ensemble_fully_supervised_localization
from utils.utils import compute_mean_iou, compute_mean_f1, compute_mean_ap, compute_mean_acc_detection, compute_mean_ap_detection

In [ ]:
# set seed for reproducibility
SEED = 0
def set_seed():
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

# save image predictions to files
def save_image_localization_scores_to_file(ious, f1_best, f1_fixed, aps, img_paths, params, dataset_name):
    with open(params.save_dir_results + f"/scores_{dataset_name}.txt", 'a') as f:
        f.write(f'image path \t iou \t f1_best \t f1_fixed \t ap\n')
        for iou, f1_b, f1_f, ap, img_path in zip(ious, f1_best, f1_fixed, aps, img_paths):
            f.write(f'{img_path} \t {iou} \t {f1_b} \t {f1_f} \t {ap}\n')

# constants for image processing
MEAN = {
    "imagenet":[0.485, 0.456, 0.406],
    "clip":[0.48145466, 0.4578275, 0.40821073]
}
STD = {
    "imagenet":[0.229, 0.224, 0.225],
    "clip":[0.26862954, 0.26130258, 0.27577711]
}

In [ ]:
new_dataset_name = 'train_all_4_datasets'

zip_path = f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets_zip/combined_training_for_ood/{new_dataset_name}.zip'
new_content_root_path = f'/content/datasets/dolos_data/celebahq/fake/'

os.makedirs(new_content_root_path, exist_ok=True)

# move the dataset from drive to /content (SSD) for better performance in I/O
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_content_root_path)

In [ ]:
# set parameters
params = Parameters()

# set the experiment name and output directory
# alert if the experiment name is not set or the output directory already exists
params.experiment_name = 'test_2_step_model'
assert params.experiment_name != '', 'Please set the experiment name'
experiment_path = f'/content/experiments/{params.experiment_name}'
if os.path.exists(experiment_path):
    shutil.rmtree(experiment_path)
params.create_output_dirs()

# set data labels to test
params.data_label = 'test'

# set the model checkpoint path
# params.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/test_repaint/best_localization_model_vit+rn50_conv12_iou_65.4029_test_repaint.pth'
# assert params.checkpoint_path != '', 'Please set the checkpoint path'

# set model parameters for testing
# state_dict = torch.load(params.checkpoint_path, map_location='cpu')
# params.decoder_type = state_dict['decoder_type']
# params.feature_layer = state_dict['feature_layer']

# params.arch = 'CLIP:ViT-L/14,RN50'
# params.arch = 'CLIP:RN50'

params.task_type = 'fully_supervised_localization'

# params.test_real_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets/dolos_data/celebahq/'
save_dir_res = params.save_dir_results

# set the batch size and num threads
params.batch_size = 64
params.num_threads = 8

# for sam models
params.sam_checkpoint_path = '../checkpoints/sam2.1_hiera_small.pt'
params.sam_config_path = 'configs/sam2.1/sam2.1_hiera_s.yaml'

In [ ]:
params_classifier = copy.deepcopy(params)
params_classifier.arch = 'CLIP:ViT-L/14'
params_classifier.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/classification/best_classification_model_ap_0.8310_all_4.pth'
state_dict_classifier = torch.load(params_classifier.checkpoint_path, map_location='cpu')
params_classifier.decoder_type = state_dict_classifier['decoder_type']
params_classifier.feature_layer = state_dict_classifier['feature_layer']
params_classifier.task_type = 'classification'
model_classifier = get_detection_model(params_classifier)
params_classifier.task_type = 'fully_supervised_localization'
model_classifier.load_state_dict(state_dict_classifier['model'], strict=False)
model_classifier = model_classifier.cuda()
model_classifier.eval()


params_ldm = copy.deepcopy(params)
params_ldm.arch = 'CLIP:ViT-L/14,RN50'
params_ldm.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/vit+rn50/best_localization_model_vit+rn50_conv12_iou_51.3407_ldm.pth'
state_dict_ldm = torch.load(params_ldm.checkpoint_path, map_location='cpu')
params_ldm.decoder_type = state_dict_ldm['decoder_type']
params_ldm.feature_layer = state_dict_ldm['feature_layer']
model_ldm = get_model(params_ldm)
model_ldm.load_state_dict(state_dict_ldm['model'], strict=False)
model_ldm = model_ldm.cuda()
model_ldm.eval()

params_repaint = copy.deepcopy(params)
params_repaint.arch = 'CLIP:ViT-L/14,RN50'
params_repaint.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/vit+rn50/best_localization_model_vit+rn50_conv12_iou_73.5066_repaint.pth'
state_dict_repaint = torch.load(params_repaint.checkpoint_path, map_location='cpu')
params_repaint.decoder_type = state_dict_repaint['decoder_type']
params_repaint.feature_layer = state_dict_repaint['feature_layer']
model_repaint = get_model(params_repaint)
model_repaint.load_state_dict(state_dict_repaint['model'], strict=False)
model_repaint = model_repaint.cuda()
model_repaint.eval()

params_lama = copy.deepcopy(params)
params_lama.arch = 'SAM2'
# params_lama.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/vit+rn50/best_localization_model_vit+rn50_conv12_iou_85.2398_lama.pth'
params_lama.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/SAM-DeepfakeLocalization-local/trained_models/train_new/lama/high_res_feats_0_32_64/best_localization_model_iou_92.7912.pth'
state_dict_lama = torch.load(params_lama.checkpoint_path, map_location='cpu')
params_lama.decoder_type = state_dict_lama['decoder_type']
params_lama.feature_layer = state_dict_lama['feature_layer']
# model_lama = get_model(params_lama)
model_lama_sam = get_sam_model(params_lama)
model_lama_sam.load_state_dict(state_dict_lama['model'], strict=False)
model_lama_sam = model_lama_sam.cuda()
model_lama_sam.eval()

params_pluralistic = copy.deepcopy(params)
params_pluralistic.arch = 'SAM2'
# params_pluralistic.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/vit+rn50/best_localization_model_vit+rn50_conv12_iou_83.1698_pluralistic.pth'
params_pluralistic.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/SAM-DeepfakeLocalization-local/trained_models/train_new/pluralistic/high_res_feats_0_32_64/best_localization_model_iou_91.2513.pth'
state_dict_pluralistic = torch.load(params_pluralistic.checkpoint_path, map_location='cpu')
params_pluralistic.decoder_type = state_dict_pluralistic['decoder_type']
params_pluralistic.feature_layer = state_dict_pluralistic['feature_layer']
model_pluralistic_sam = get_sam_model(params_pluralistic)
model_pluralistic_sam.load_state_dict(state_dict_pluralistic['model'], strict=False)
model_pluralistic_sam = model_pluralistic_sam.cuda()
model_pluralistic_sam.eval()

# sam got the best results on GAN-based manipulations
localization_models_sam = {
    'ldm': model_ldm,
    'repaint': model_repaint,
    'lama': model_lama_sam,
    'pluralistic': model_pluralistic_sam
}

# now let's illustrate the clip models as well
params_lama = copy.deepcopy(params)
params_lama.arch = 'CLIP:ViT-L/14,RN50'
params_lama.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/vit+rn50/best_localization_model_vit+rn50_conv12_iou_85.2398_lama.pth'
state_dict_lama = torch.load(params_lama.checkpoint_path, map_location='cpu')
params_lama.decoder_type = state_dict_lama['decoder_type']
params_lama.feature_layer = state_dict_lama['feature_layer']
model_lama_clip = get_model(params_lama)
model_lama_clip.load_state_dict(state_dict_lama['model'], strict=False)
model_lama_clip = model_lama_clip.cuda()
model_lama_clip.eval()

params_pluralistic = copy.deepcopy(params)
params_pluralistic.arch = 'CLIP:ViT-L/14,RN50'
params_pluralistic.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/vit+rn50/best_localization_model_vit+rn50_conv12_iou_83.1698_pluralistic.pth'
state_dict_pluralistic = torch.load(params_pluralistic.checkpoint_path, map_location='cpu')
params_pluralistic.decoder_type = state_dict_pluralistic['decoder_type']
params_pluralistic.feature_layer = state_dict_pluralistic['feature_layer']
model_pluralistic_clip = get_model(params_pluralistic)
model_pluralistic_clip.load_state_dict(state_dict_pluralistic['model'], strict=False)
model_pluralistic_clip = model_pluralistic_clip.cuda()
model_pluralistic_clip.eval()


localization_models_clip = {
    'ldm': model_ldm,
    'repaint': model_repaint,
    'lama': model_lama_clip,
    'pluralistic': model_pluralistic_clip
}

In [ ]:
params_detector = copy.deepcopy(params)
params_detector.arch = 'CLIP:ViT-L/14'
params_detector.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/detection/best_detection_model_ap_0.8992_all_4.pth' # detector trained on all 4
state_dict_detector = torch.load(params_detector.checkpoint_path, map_location='cpu')
params_detector.decoder_type = state_dict_detector['decoder_type']
params_detector.feature_layer = state_dict_detector['feature_layer']
params_detector.task_type = 'detection'
model_detector = get_detection_model(params_detector)
params_detector.task_type = 'fully_supervised_localization'
model_detector.load_state_dict(state_dict_detector['model'], strict=False)
model_detector = model_detector.cuda()
model_detector.eval()


params_combined_training_all_4 = copy.deepcopy(params)
params_combined_training_all_4.arch = 'CLIP:ViT-L/14,RN50'
params_combined_training_all_4.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/train_all_4/best_localization_model_vit+rn50_conv12_iou_63.5034.pth'
state_dict_combined_training_all_4 = torch.load(params_combined_training_all_4.checkpoint_path, map_location='cpu')
params_combined_training_all_4.decoder_type = state_dict_combined_training_all_4['decoder_type']
params_combined_training_all_4.feature_layer = state_dict_combined_training_all_4['feature_layer']
model_combined_training_all_4 = get_model(params_combined_training_all_4)
model_combined_training_all_4.load_state_dict(state_dict_combined_training_all_4['model'], strict=False)
model_combined_training_all_4 = model_combined_training_all_4.cuda()
model_combined_training_all_4.eval()

CLIPModelLocalisation(
  (fc): Sequential(
    (0): Conv2d(2048, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Upsample(scale_factor=2.0, mode='bilinear')
    (10): Conv2d(512, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (14): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
two_step_model_classifier_sam = TwoStepModel(model_classifier, localization_models_sam, first_step_type='classifier')

two_step_model_classifier_clip = TwoStepModel(model_classifier, localization_models_clip, first_step_type='classifier')

two_step_model_detector = TwoStepModel(model_detector, model_combined_training_all_4, first_step_type='detector')

two_step_models = {
    'detector': two_step_model_detector,
    'classifier_sam': two_step_model_classifier_sam,
    'classifier_clip': two_step_model_classifier_clip
}

In [ ]:
# prepare the datasets and the results file
if params.task_type == 'fully_supervised_localization':
    dataset_paths = LOCALIZATION_DATASET_PATHS
    with open(os.path.join(params.save_dir_results, 'scores.txt'), 'a') as f:
        f.write('dataset \t iou \t f1_best \t f1_fixed \t ap \n')
elif params.task_type == 'detection':
    dataset_paths = DETECTION_DATASET_PATHS
    with open(os.path.join(params.save_dir_results, 'scores.txt'), 'a') as f:
        f.write('dataset \t ap \t acc_fixed_thresh \t acc_best_thresh \t best_threshold \n')

In [ ]:
params.batch_size = 32
params.num_threads = 4

In [ ]:
# first test will include real images, so that both two-step models can be used

# we will use the specific test sets first (adding real images)

for first_step_type in two_step_models.keys():
    params.save_dir_results = f'{save_dir_res}_2_step_model_{first_step_type}_on_specific_datasets+real_images'
    two_step_model = two_step_models[first_step_type]
    print(f'Testing Two Step Model with first step type: {first_step_type} on specific datasets + real images')

    for dataset_path in dataset_paths:
        if 'autosplice' in dataset_path['key']:
            continue
        print(f"Testing on {dataset_path['key']}")
        set_seed()
        os.makedirs(os.path.join(params.save_dir_results, dataset_path['key']), exist_ok=True)

        params.train_dataset = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['key'])
        params.test_fake_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['fake_path'])
        params.test_masks_ground_truth_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['masks_path'])
        params.test_real_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['real_path'])

        params.task_type = 'fully_supervised_localization_with_real_images'
        # if 'sam' in first_step_type and (dataset_path['key'] == 'lama' or dataset_path['key'] == 'pluralistic'):
        #     prev_arch = params.arch
        #     params.arch = 'SAM2'
        data_loader = get_dataloader(params)
        # if 'sam' in first_step_type and (dataset_path['key'] == 'lama' or dataset_path['key'] == 'pluralistic'):
        #     params.arch = prev_arch
        params.task_type = 'fully_supervised_localization'

        # create the directory for dataset results
        os.makedirs(os.path.join(params.save_dir_results, params.train_dataset), exist_ok=True)

        if params.task_type == 'fully_supervised_localization':
            ious, f1_best, f1_fixed, ap, original_img_paths = validate_ensemble_fully_supervised_localization(
                two_step_model, data_loader, dataset_path['key'], params.save_dir_results
            )
            save_image_localization_scores_to_file(ious, f1_best, f1_fixed, ap, original_img_paths, params, dataset_path['key'])

            mean_iou = compute_mean_iou(ious, verbose=True, extra_text=dataset_path['key'])
            mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=dataset_path['key'])
            mean_ap = compute_mean_ap(ap, verbose=True, extra_text=dataset_path['key'])

            with open(os.path.join(params.save_dir_results, f"scores_{dataset_path['key']}.txt"), 'a') as f:
                f.write(f"{dataset_path['key']} \t {mean_iou:.4f} \t {mean_f1_best:.4f} \t {mean_f1_fixed:.4f} \t {mean_ap:.4f} \n")

        print()


Testing Two Step Model with first step type: detector on specific datasets + real images
Testing on lama
Length of dataset:  1800


  0%|          | 0/57 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples found in target, recall is undefined. Setting recall to one for all thresholds.
  warnings.warn(*args, **kwargs)
100%|██████████| 57/57 [02:20<00:00,  2.46s/it]


lamaMean IOU: 78.63
lamaMean F1 best: 0.8689
lamaMean F1 fixed: 0.8436
lamaMean AP: 0.4557

Testing on ldm
Length of dataset:  1800


100%|██████████| 57/57 [02:19<00:00,  2.45s/it]


ldmMean IOU: 63.11
ldmMean F1 best: 0.7417
ldmMean F1 fixed: 0.6727
ldmMean AP: 0.3314

Testing on pluralistic
Length of dataset:  1800


...

ldmMean IOU: 72.42
ldmMean F1 best: 0.8273
ldmMean F1 fixed: 0.7646
ldmMean AP: 0.349

Testing on pluralistic
Length of dataset:  1800


100%|██████████| 57/57 [02:13<00:00,  2.34s/it]


pluralisticMean IOU: 82.37
pluralisticMean F1 best: 0.8953
pluralisticMean F1 fixed: 0.8862
pluralisticMean AP: 0.4005

Testing on repaint-p2-9k
Length of dataset:  1800


100%|██████████| 57/57 [02:05<00:00,  2.19s/it]

repaint-p2-9kMean IOU: 64.5
repaint-p2-9kMean F1 best: 0.7186
repaint-p2-9kMean F1 fixed: 0.7054
repaint-p2-9kMean AP: 0.2243



In [ ]:
# now we can test on all test sets at once

for first_step_type in two_step_models.keys():
    params.save_dir_results = f'{save_dir_res}_2_step_model_{first_step_type}_on_all_datasets'
    two_step_model = two_step_models[first_step_type]
    print(f'Testing Two Step Model with first step type: {first_step_type} on all datasets')

    dataset_path_all_4 = dict(
            fake_path=f'datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/test',
            real_path=f'datasets/dolos_data/celebahq/real/test',
            masks_path=f'datasets/dolos_data/celebahq/fake/train_all_4_datasets/masks/test',
            key='train_all_4_datasets'
        )

    print(f"Testing on {dataset_path_all_4['key']}")
    set_seed()
    os.makedirs(os.path.join(params.save_dir_results, dataset_path_all_4['key']), exist_ok=True)

    params.train_dataset = os.path.join('/content/', dataset_path_all_4['key'])
    params.test_fake_path = os.path.join('/content/', dataset_path_all_4['fake_path'])
    params.test_masks_ground_truth_path = os.path.join('/content/', dataset_path_all_4['masks_path'])
    params.test_real_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path_all_4['real_path'])

    params.task_type = 'fully_supervised_localization_with_real_images'
    # if 'sam' in first_step_type and (dataset_path['key'] == 'lama' or dataset_path['key'] == 'pluralistic'):
    #         prev_arch = params.arch
    #         params.arch = 'SAM2'
    data_loader = get_dataloader(params)
    # if 'sam' in first_step_type and (dataset_path['key'] == 'lama' or dataset_path['key'] == 'pluralistic'):
    #     params.arch = prev_arch
    params.task_type = 'fully_supervised_localization'

    if params.task_type == 'fully_supervised_localization':
        ious, f1_best, f1_fixed, ap, original_img_paths = validate_ensemble_fully_supervised_localization(
            two_step_model, data_loader, dataset_path_all_4['key'], params.save_dir_results
        )
        save_image_localization_scores_to_file(ious, f1_best, f1_fixed, ap, original_img_paths, params, dataset_path_all_4['key'])

        mean_iou = compute_mean_iou(ious, verbose=True, extra_text=dataset_path_all_4['key'])
        mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=dataset_path_all_4['key'])
        mean_ap = compute_mean_ap(ap, verbose=True, extra_text=dataset_path_all_4['key'])

        with open(os.path.join(params.save_dir_results, f"scores_{dataset_path_all_4['key']}.txt"), 'a') as f:
            f.write(f"{dataset_path_all_4['key']} \t {mean_iou:.4f} \t {mean_f1_best:.4f} \t {mean_f1_fixed:.4f} \t {mean_ap:.4f} \n")

    print()


Testing Two Step Model with first step type: detector on all datasets
Testing on train_all_4_datasets
Length of dataset:  4500


100%|██████████| 141/141 [05:52<00:00,  2.50s/it]


train_all_4_datasetsMean IOU: 61.86
train_all_4_datasetsMean F1 best: 0.7637
train_all_4_datasetsMean F1 fixed: 0.7109
train_all_4_datasetsMean AP: 0.5855

Testing Two Step Model with first step type: classifier_sam on all datasets
Testing on train_all_4_datasets
Length of dataset:  4500


100%|██████████| 141/141 [06:25<00:00,  2.73s/it]


train_all_4_datasetsMean IOU: 69.09
train_all_4_datasetsMean F1 best: 0.7886
train_all_4_datasetsMean F1 fixed: 0.7524
train_all_4_datasetsMean AP: 0.5868

Testing Two Step Model with first step type: classifier_clip on all datasets
Testing on train_all_4_datasets
Length of dataset:  4500


100%|██████████| 141/141 [05:47<00:00,  2.47s/it]

train_all_4_datasetsMean IOU: 66.61
train_all_4_datasetsMean F1 best: 0.7907
train_all_4_datasetsMean F1 fixed: 0.7538
train_all_4_datasetsMean AP: 0.5824



In [ ]:
# second test will only use deepfake images, therefore we are only interested in using the classifier two-step model (to compare the 4 specialists with 1 generalist - which doesn't need a classifier to specify the type of the deepfake)
# indeed, we could have used a classifier that only saw fake images (and learned to distinguish between them, but that would be quite specific, we would rather want to classify real images as well)

# here we will use the 4 datasets directly

for first_step_type in two_step_models.keys():
    if first_step_type == 'detector':
        continue
    params.save_dir_results = f'{save_dir_res}_2_step_model_{first_step_type}_on_specific_datasets'
    two_step_model = two_step_models[first_step_type]
    print(f'Testing Two Step Model with first step type: {first_step_type} on specific datasets')

    for dataset_path in dataset_paths:
        if 'autosplice' in dataset_path['key']:
            continue
        print(f"Testing on {dataset_path['key']}")
        set_seed()
        os.makedirs(os.path.join(params.save_dir_results, dataset_path['key']), exist_ok=True)

        params.train_dataset = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['key'])
        params.test_fake_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['fake_path'])
        params.test_masks_ground_truth_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['masks_path'])
        params.test_real_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['real_path'])

        # if 'sam' in first_step_type and (dataset_path['key'] == 'lama' or dataset_path['key'] == 'pluralistic'):
            # prev_arch = params.arch
            # params.arch = 'SAM2'
        data_loader = get_dataloader(params)
        # if 'sam' in first_step_type and (dataset_path['key'] == 'lama' or dataset_path['key'] == 'pluralistic'):
            # params.arch = prev_arch

        # create the directory for dataset results
        os.makedirs(os.path.join(params.save_dir_results, params.train_dataset), exist_ok=True)

        if params.task_type == 'fully_supervised_localization':
            ious, f1_best, f1_fixed, ap, original_img_paths = validate_ensemble_fully_supervised_localization(
                two_step_models[first_step_type], data_loader, dataset_path['key'], params.save_dir_results
            )
            save_image_localization_scores_to_file(ious, f1_best, f1_fixed, ap, original_img_paths, params, dataset_path['key'])

            mean_iou = compute_mean_iou(ious, verbose=True, extra_text=dataset_path['key'])
            mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=dataset_path['key'])
            mean_ap = compute_mean_ap(ap, verbose=True, extra_text=dataset_path['key'])

            with open(os.path.join(params.save_dir_results, f"scores_{dataset_path['key']}.txt"), 'a') as f:
                f.write(f"{dataset_path['key']} \t {mean_iou:.4f} \t {mean_f1_best:.4f} \t {mean_f1_fixed:.4f} \t {mean_ap:.4f} \n")

        print()

Testing Two Step Model with first step type: classifier_sam on specific datasets
Testing on lama
Length of dataset:  900


100%|██████████| 29/29 [01:39<00:00,  3.43s/it]


lamaMean IOU: 91.49
lamaMean F1 best: 0.9646
lamaMean F1 fixed: 0.953
lamaMean AP: 0.9808

Testing on ldm
Length of dataset:  900


100%|██████████| 29/29 [01:23<00:00,  2.86s/it]


ldmMean IOU: 49.66
ldmMean F1 best: 0.7032
ldmMean F1 fixed: 0.577
ldmMean AP: 0.6966

Testing on pluralistic
Length of dataset:  900


...

ldmMean IOU: 49.84
ldmMean F1 best: 0.7046
ldmMean F1 fixed: 0.5793
ldmMean AP: 0.6981

Testing on pluralistic
Length of dataset:  900


100%|██████████| 29/29 [01:18<00:00,  2.71s/it]


pluralisticMean IOU: 69.74
pluralisticMean F1 best: 0.8406
pluralisticMean F1 fixed: 0.8224
pluralisticMean AP: 0.801

Testing on repaint-p2-9k
Length of dataset:  900


100%|██████████| 29/29 [01:10<00:00,  2.44s/it]

repaint-p2-9kMean IOU: 34.0
repaint-p2-9kMean F1 best: 0.4872
repaint-p2-9kMean F1 fixed: 0.4607
repaint-p2-9kMean AP: 0.4486



In [ ]:
import shutil
from google.colab import files

for folder in os.listdir(f'/content/experiments/{params.experiment_name}'):
    shutil.make_archive(f'{folder}', 'zip', f'/content/experiments/{params.experiment_name}/{folder}')
    files.download(f'results_{folder}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>